In [ ]:
import os
import torch as tr
import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
from pathlib import Path

from vre.representations import build_representations_from_cfg
from vre.utils import get_project_root
from vre import FFmpegVideo
from vre_repository import get_vre_repository
from vre_repository.optical_flow.raft import FlowRaft

device = "cuda" if tr.cuda.is_available() else "cpu"

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
video = FFmpegVideo(get_project_root() / "resources/test_video.mp4")
print(video.shape, video.fps)

In [ ]:
# os.environ["VRE_DEVICE"] = device = "cuda" if tr.cuda.is_available() else "cpu"
# all_representations_dict = OmegaConf.load(open(Path.cwd() / "cfg.yaml", "r"))

# device = "cuda" if tr.cuda.is_available() else "cpu"

# representations = build_representations_from_cfg(all_representations_dict, representation_types=get_vre_repository())
# name_to_repr = {r.name: r for r in representations}
# print(representations)

## Run the representations for two particular frame

In [ ]:
# # inference setup (this is done inside VRE's main loop at run() as well)
# depth, normals = name_to_repr["depth_marigold"], name_to_repr["normals_svd(depth_marigold)"]
# depth.vre_setup() if depth.setup_called is False else None

# np.random.seed(43)
# mb = 2
# ixs = sorted([np.random.randint(0, len(video) - 1) for _ in range(mb)])
# print(ixs)

# depth.data = normals.data = None
# depth.compute(video, ixs)
# normals.compute(video, ixs)
# y_depth_img = depth.make_images(depth.data)
# y_normals_img = normals.make_images(normals.data)
# for i in range(mb):
#     fig, ax = plt.subplots(1, 3, figsize=(20, 10))
#     ax[0].imshow(depth.data.frames[i])
#     ax[1].imshow(y_depth_img[i])
#     ax[2].imshow(y_normals_img[i])
#     plt.show()
# depth.vre_free()

### Optical flow +/-1

In [ ]:
h, w = video.shape[1:3]
# h, w = [540, 960]
print(h, w)
flow = FlowRaft(name="flow_raft", dependencies=[], inference_width=w, inference_height=h, iters=5,
                small=False, delta=1)
flow_l = FlowRaft(name="flow_raft", dependencies=[], inference_width=w, inference_height=h, iters=5,
                small=False, delta=-1)
flow.device = flow_l.device = device
print(flow)
flow.vre_setup() if flow.setup_called is False else None
flow_l.vre_setup() if flow_l.setup_called is False else None

# np.random.seed(43)
mb = 2
ixs = sorted([np.random.randint(0, len(video) - 1) for _ in range(mb)])
print(ixs)

flow.data = flow_l.data = None
flow.compute(video, ixs)
flow_l.compute(video, ixs)
print(flow.data.output.reshape(-1, 2).mean(0) * [h, w], flow.data.output.reshape(-1, 2).std(0))
print(flow_l.data.output.reshape(-1, 2).mean(0)  * [h, w], flow_l.data.output.reshape(-1, 2).std(0))
flow_img = flow.make_images(flow.data)
flow_l_img = flow_l.make_images(flow_l.data)
for i in range(mb):
    fig, ax = plt.subplots(1, 3, figsize=(20, 10))
    # assert (flow.data.output[i] - flow_l.data.output[i]).__abs__().mean() > 1e-3
    ax[0].imshow(video[ixs[i]])
    ax[1].imshow(flow_img[i])
    ax[2].imshow(flow_l_img[i])
plt.show()
flow.vre_free()


In [ ]:
flow.data.output